<a href="https://colab.research.google.com/github/anand1994sp/MACHINE-LEARNING-PROJECTS/blob/main/Random_Forest_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [58]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from math import radians,cos,sin,sqrt,asin
plt.rcParams["figure.figsize"]=[15,8]

In [59]:
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn import metrics
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor

In [60]:
df=pd.read_csv("/content/TaxiFare.csv",parse_dates=["date_time_of_pickup"])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype              
---  ------                --------------  -----              
 0   unique_id             50000 non-null  object             
 1   amount                50000 non-null  float64            
 2   date_time_of_pickup   50000 non-null  datetime64[ns, UTC]
 3   longitude_of_pickup   50000 non-null  float64            
 4   latitude_of_pickup    50000 non-null  float64            
 5   longitude_of_dropoff  50000 non-null  float64            
 6   latitude_of_dropoff   50000 non-null  float64            
 7   no_of_passenger       50000 non-null  int64              
dtypes: datetime64[ns, UTC](1), float64(5), int64(1), object(1)
memory usage: 3.1+ MB


In [61]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
amount,50000.0,11.364171,9.685557,-5.000000,6.000000,8.500000,12.500000,200.000000
longitude_of_pickup,50000.0,-72.509756,10.393860,-75.423848,-73.992062,-73.981840,-73.967148,40.783472
latitude_of_pickup,50000.0,39.933759,6.224857,-74.006893,40.734880,40.752678,40.767360,401.083332
longitude_of_dropoff,50000.0,-72.504616,10.407570,-84.654241,-73.991152,-73.980082,-73.963584,40.851027
latitude_of_dropoff,50000.0,39.926251,6.014737,-74.006377,40.734372,40.753372,40.768167,43.415190
no_of_passenger,50000.0,1.667840,1.289195,0.000000,1.000000,1.000000,2.000000,6.000000


In [62]:
df.dtypes

unique_id                            object
amount                              float64
date_time_of_pickup     datetime64[ns, UTC]
longitude_of_pickup                 float64
latitude_of_pickup                  float64
longitude_of_dropoff                float64
latitude_of_dropoff                 float64
no_of_passenger                       int64
dtype: object

In [63]:
df.drop("unique_id",inplace=True,axis=1)

In [64]:
import datetime as dt
df=df.assign(hour=df.date_time_of_pickup.dt.hour,
             day=df.date_time_of_pickup.dt.day,
             month=df.date_time_of_pickup.dt.month,
             year=df.date_time_of_pickup.dt.year,
             dayofweek=df.date_time_of_pickup.dt.dayofweek)
df

,amount,date_time_of_pickup,longitude_of_pickup,latitude_of_pickup,longitude_of_dropoff,latitude_of_dropoff,no_of_passenger,hour,day,month,year,dayofweek
0,4.5,2009-06-15 17:26:21+00:00,-73.844311,40.721319,-73.841610,40.712278,1,17,15,6,2009,0
1,16.9,2010-01-05 16:52:16+00:00,-74.016048,40.711303,-73.979268,40.782004,1,16,5,1,2010,1
2,5.7,2011-08-18 00:35:00+00:00,-73.982738,40.761270,-73.991242,40.750562,2,0,18,8,2011,3
3,7.7,2012-04-21 04:30:42+00:00,-73.987130,40.733143,-73.991567,40.758092,1,4,21,4,2012,5
4,5.3,2010-03-09 07:51:00+00:00,-73.968095,40.768008,-73.956655,40.783762,1,7,9,3,2010,1
...,...,...,...,...,...,...,...,...,...,...,...,...
49995,15.0,2013-06-12 23:25:15+00:00,-73.999973,40.748531,-74.016899,40.705993,1,23,12,6,2013,2
49996,7.5,2015-06-22 17:19:18+00:00,-73.984756,40.768211,-73.987366,40.760597,1,17,22,6,2015,0
49997,6.9,2011-01-30 04:53:00+00:00,-74.002698,40.739428,-73.998108,40.759483,1,4,30,1,2011,6
49998,4.5,2012-11-06 07:09:00+00:00,-73.946062,40.777567,-73.953450,40.779687,2,7,6,11,2012,1


In [65]:
df.drop("date_time_of_pickup",inplace=True,axis=1)

In [66]:
df.shape

(50000, 11)

In [67]:
#df.drop(df[df["no_of_passenger"]==0].index,inplace=True)
#df.shape

In [68]:
y=pd.DataFrame(df["amount"])
x=df.drop("amount",axis=1)

In [69]:
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.3,random_state=4)

In [70]:
rf=RandomForestRegressor(n_estimators=100,random_state=5)

In [71]:
rf.fit(xtrain,ytrain)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestRegressor(random_state=5)

In [72]:
ypred=rf.predict(xtest)

In [73]:
MAE=metrics.mean_absolute_error(ytest,ypred)
MAE

2.124042151111111

In [74]:
MSE=metrics.mean_squared_error(ytest,ypred)
MSE

19.03292519660752

In [76]:
RMSE=np.sqrt(metrics.mean_squared_error(ytest,ypred))
RMSE

4.362674087828188

In [77]:
tuned_parameters=[{"n_estimator":[85,100],
                   "min_sample_split":[15,20],
                   "max_depth":[8,10],
                   "min_sample_leaf":[5,10],
                   "max_leaf_nodes":[10,15]}]

In [ ]:
rf2=RandomForestRegressor(randomstate=10)